NumPy package contains an iterator object numpy.nditer. It is an efficient multidimensional iterator object using which it is possible to iterate over an array. Each element of an array is visited using Python’s standard Iterator interface.

    class numpy.nditer(op, [flags], [op_flags], [op_dtypes], [order], [casting], [op_axes], [itershape], [buffersize])[link text]

op = The array(s) to iterate over.

flags = Flags to control the behavior of the iterator.

op_flags = This is a list of flags for each operand. At minimum, one of readonly, readwrite, or writeonly must be specified.

op_dtypes = The required data type(s) of the operands. If copying or buffering is enabled, the data will be converted to/from their original types.

order = {‘C’, ‘F’, ‘A’, ‘K’}; Default K.

casting = Controls what kind of data casting may occur when making a copy or buffering. Setting this to ‘unsafe’ is not recommended, as it can adversely affect accumulations.

    ‘no’ means the data types should not be cast at all.

    ‘equiv’ means only byte-order changes are allowed.

    ‘safe’ means only casts which can preserve values are allowed.

    ‘same_kind’ means only safe casts or casts within a kind, like float64 to float32, are allowed.

    ‘unsafe’ means any data conversions may be done.

op_axes = If provided, is a list of ints or None for each operands. The list of axes for an operand is a mapping from the dimensions of the iterator to the dimensions of the operand. A value of -1 can be placed for entries, causing that dimension to be treated as newaxis.

itershape = tuple of ints; The desired shape of the iterator. This allows allocate operands with a dimension mapped by op_axes not corresponding to a dimension of a different operand to get a value not equal to 1 for that dimension.

buffersize = When buffering is enabled, controls the size of the temporary buffers. Set to 0 for the default value.

[Link to Doc](https://numpy.org/doc/stable/reference/arrays.nditer.html)

### Single Array iteration

In [1]:
import numpy as np

a = np.arange(6).reshape(2,3)
print(a)
print()

for x in np.nditer(a):
    print(x, end=' ')

[[0 1 2]
 [3 4 5]]

0 1 2 3 4 5 

### Modifying Array values

By default, the nditer treats the input operand as a read-only object. To be able to modify the array elements, you must specify either read-write or write-only mode using the ‘readwrite’ or ‘writeonly’ per-operand flags.

The nditer will then yield writeable buffer arrays which you may modify. However, because the nditer must copy this buffer data back to the original array once iteration is finished, you must signal when the iteration is ended, by one of two methods. You may either:

used the nditer as a context manager using the with statement, and the temporary data will be written back when the context is exited.

call the iterator’s close method once finished iterating, which will trigger the write-back.

The nditer can no longer be iterated once either close is called or its context is exited.

In [4]:
a = np.arange(6).reshape(2,3)
print(a)
print()

with np.nditer(a, op_flags=['readwrite']) as it:
   for x in it:
       x[...] = 2 * x

print(a)

[[0 1 2]
 [3 4 5]]

[[ 0  2  4]
 [ 6  8 10]]


### Tracking an Index or Multi-Index

During iteration, you may want to use the index of the current element in a computation. For example, you may want to visit the elements of an array in memory order, but use a C-order, Fortran-order, or multidimensional index to look up values in a different array.

The index is tracked by the iterator object itself, and accessible through the index or multi_index properties, depending on what was requested.

In [11]:
a = np.arange(6).reshape(2,3)
it = np.nditer(a, flags=['f_index'])
for x in it:
    print("%d <%d>" % (x, it.index), end=' ')

0 <0> 1 <2> 2 <4> 3 <1> 4 <3> 5 <5> 

In [12]:
it = np.nditer(a, flags=['multi_index'])
for x in it:
    print("%d <%s>" % (x, it.multi_index), end=' ')


0 <(0, 0)> 1 <(0, 1)> 2 <(0, 2)> 3 <(1, 0)> 4 <(1, 1)> 5 <(1, 2)> 

### Broadcasting Array Iteration

NumPy has a set of rules for dealing with arrays that have differing shapes which are applied whenever functions take multiple operands which combine element-wise. This is called broadcasting. The nditer object can apply these rules for you when you need to write such a function.

In [ ]:
a = np.arange(3)
b = np.arange(6).reshape(2,3)
for x, y in np.nditer([a,b]):
    print("%d:%d" % (x,y), end=' ') 